# Spark DataFrame Basics

Spark DataFrames are the workhouse and main way of working with Spark and Python post Spark 2.0. DataFrames act as powerful versions of tables, with rows and columns, easily handling large datasets. The shift to DataFrames provides many advantages:
* A much simpler syntax
* Ability to use SQL directly in the dataframe
* Operations are automatically distributed across RDDs
    
If you've used R or even the pandas library with Python you are probably already familiar with the concept of DataFrames. Spark DataFrame expand on a lot of these concepts, allowing you to transfer that knowledge easily by understanding the simple syntax of Spark DataFrames. Remember that the main advantage to using Spark DataFrames vs those other programs is that Spark can handle data across many RDDs, huge data sets that would never fit on a single computer. That comes at a slight cost of some "peculiar" syntax choices, but after this course you will feel very comfortable with all those topics!

Let's get started!

## Creating a DataFrame

First we need to start a SparkSession:

In [0]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [0]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()

You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [0]:
# We'll discuss how to read other options later.
# This dataset is from Spark's examples

# read the people.json
df = spark.read.json("/FileStore/tables/people.json")

#### Showing the data

In [0]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [0]:
df.columns

Out[6]: ['age', 'name']

In [0]:
df.describe()

Out[7]: DataFrame[summary: string, age: string, name: string]

Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [0]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [0]:
# make age integertype and name stringtype
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [0]:
final_struc = StructType(fields=data_schema)

In [0]:
# read people.json again
df = spark.read.json("/FileStore/tables/people.json", schema=final_struc)

In [0]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing the data

In [0]:
# grab the age
df['age']

Out[13]: Column<'age'>

In [0]:
type(df['age'])

Out[15]: pyspark.sql.column.Column

In [0]:
type(df['age'])

Out[16]: pyspark.sql.column.Column

In [0]:
df.select('age')

Out[17]: DataFrame[age: int]

In [0]:
type(df.select('age'))

Out[18]: pyspark.sql.dataframe.DataFrame

In [0]:
# Returns list of Row objects
df.head(2)

Out[19]: [Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Multiple Columns:

In [0]:
 #grab age and name
df.select(['name','name'])

Out[21]: DataFrame[name: string, name: string]

In [0]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [0]:
# rename column age to newage
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [0]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# rename age to supernewage
df.withColumnRenamed('supernewage','age').show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



More complicated operations to create new columns

In [0]:
# select and double the age and rename to doubleage
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [0]:
# add one to age and name it add_one_age
df.withColumn('add_one_age',df["age"]).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         30|
|  19| Justin|         19|
+----+-------+-----------+



In [0]:
# half the age and save to half_age
df.withColumn('half_age',df['age']).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|      30|
|  19| Justin|      19|
+----+-------+--------+



In [0]:
df.withColumn('age',df['age'])

Out[29]: DataFrame[age: int, name: string]

We'll discuss much more complicated operations later on!

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [0]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [0]:
# select all (*) from people
sql_results = spark.sql("SELECT * FROM people;")

In [0]:
sql_results

Out[34]: DataFrame[age: int, name: string]

In [0]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# select all people whose age is 30
spark.sql("SELECT * FROM people WHERE age=30;").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+



We won't really be focusing on using the SQL syntax for this course in general, but keep in mind it is always there for you to get you out of bind quickly with your SQL skills!

Alright that is all we need to know for now!